In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import tensorflow as tf
import cv2
from glob import glob
import matplotlib.pyplot as plt

In [4]:
IMAGE_SIZE = [224,224]

In [5]:
train_directory = "../input/100-bird-species/train"
test_directory = "../input/100-bird-species/test"
val_directory = "../input/100-bird-species/valid"

In [6]:
vgg = VGG16(input_shape = IMAGE_SIZE + [3],weights='imagenet',include_top=False)

In [7]:
for layer in vgg.layers:
    layer.trainable = False

In [8]:
folder = glob("../input/100-bird-species/train/*")
len(folder)

In [9]:
x = Flatten()(vgg.output)
prediction = Dense(len(folder),activation='softmax')(x)

In [10]:
model = Model(inputs = vgg.input, outputs = prediction)
model.summary()

In [11]:
model.compile(
loss='categorical_crossentropy',
optimizer = 'adam',
metrics = ['accuracy'])

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range=0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_Set = train_datagen.flow_from_directory(train_directory,
                                                target_size=[224,224],
                                                batch_size=32,
                                                class_mode = 'categorical')

test_Set = test_datagen.flow_from_directory(test_directory,
                                         target_size=(224,224),
                                         batch_size = 32,
                                         class_mode = 'categorical')


In [13]:
print(len(training_Set))
print(len(test_Set))

In [14]:
r = model.fit(
    training_Set,
validation_data = test_Set,
epochs = 5,
steps_per_epoch = len(training_Set),
validation_steps = len(test_Set)
)

In [15]:
#loss
plt.plot(r.history['loss'],label='train loss')
plt.plot(r.history['val_loss'],label = 'val loss')
plt.legend()
plt.show()

In [16]:
#accuracy
plt.plot(r.history['accuracy'],label='train acc')
plt.plot(r.history['val_accuracy'],label='val acc')
plt.legend()
plt.show()

In [17]:
model.save('BC.h5')

In [19]:
from keras.models import load_model
from keras.preprocessing.image import load_img,img_to_array

model1 = load_model("./BC.h5",compile=False)

In [21]:
lab = training_Set.class_indices
lab = {k:v for v,k in lab.items()}  #swapping key and values

In [22]:
def output(location):
    img = load_img(location,target_size=(224,224,3))
    img = img_to_array(img)
    img = img/255
    img = np.expand_dims(img,[0])
    answer = model1.predict(img)
    y_class = answer.argmax(axis=-1)
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res =lab[y]
    return res

In [23]:
img = "../input/100-bird-species/train/AMERICAN BITTERN/004.jpg"
pic = load_img(img,target_size=(224,224,3))
plt.imshow(pic)
output(img)

In [24]:
import tensorflow as tf
import keras as k
print(tf.__version__)
print(k.__version__)